In [ ]:
#Work Work

In [1]:
import webbrowser
import time
import json
import pandas as pd
import alpaca_trade_api as tradeapi
from alpaca_trade_api.rest import REST, TimeFrame

In [2]:
payload=pd.read_html('https://en.wikipedia.org/wiki/Nasdaq-100')
df = payload[4]
symb_list=df['Ticker'].values.tolist()

In [3]:
SEC_KEY = 'ZTqQ3wx4U4fH2rvAaN21M7rQLCq7O8qFn9ODRIXC'
PUB_KEY = 'PKGEW0G23998ZUZNR57H'
BASE_URL = 'https://paper-api.alpaca.markets' # This is the base URL for paper trading
api = tradeapi.REST(key_id= PUB_KEY, secret_key=SEC_KEY, base_url=BASE_URL) # For real trading, don't enter a base_url


In [4]:

def get_data(symb_list,frequency,start,end,limit):
    data=api.get_bars(symb_list, frequency, start,  end, limit=9999, adjustment='raw')
    pd_data=data.df
    subset=pd_data[['symbol','close']]
    return subset


def sort_data(data):
    new=data.groupby("symbol")
    nod=len(new.get_group(data["symbol"][0]))#num of days
    price_list=[]
    symbol_list=[]
    change_list=[]
    for i in range(int(len(data)/nod)):
        dif=data['close'][nod*(i+1)-1]-data['close'][i*nod]
        avg=sum(data['close'][nod*i:nod*(i+1)])/nod
        symbol_list.append(data['symbol'][nod*i])
        price_list.append(data['close'][nod*(i+1)-1])
        change_list.append(dif/avg)
    data=pd.DataFrame(list(zip(symbol_list,price_list,change_list)),columns=['symb','price','change'])
    sorted_data=data.sort_values(by=['change'])
    sorted_data = sorted_data.reset_index(drop=True)
    return sorted_data


def get_ideal_position(sorted_data,N):
    ideal_position=sorted_data.iloc[-N:]
    ideal_position = ideal_position.reset_index(drop=True)
    return ideal_position


def get_curr_position():
    position_data=api.list_positions()
    symb,qty,entry_price,curr_price=[],[],[],[]
    for p in position_data:
        p=p._raw
        symb.append(p['symbol'])
        qty.append(p['qty'])
        entry_price.append(p['avg_entry_price'])
        curr_price.append(p['current_price'])
    position_df=pd.DataFrame(list(zip(symb,qty,entry_price,curr_price)),columns=['symb','qty','entry_price','curr_price'])
    return position_df


def generate_order_df(ideal,curr,price):
    
    order_df=pd.DataFrame({'symb':[],'qty':[],'price':[]})
    for ind,row in ideal.iterrows():
        if row['symb'] in list(curr['symb']):
            index=curr.index[curr['symb']==row['symb']]
            order_df=order_df.append({'symb':row['symb'],'qty':str(int(row['qty'])-int(curr['qty'][index])),'price':row['price'] },ignore_index=True)
        else:
            order_df=order_df.append({'symb':row['symb'],'qty':row['qty'],'price':row['price'] },ignore_index=True)  
    for ind,row in curr.iterrows():
        if row['symb'] not in list(ideal['symb']):
            print(row['symb'])
            order_df=order_df.append({'symb':row['symb'],'qty':(int(row['qty'])*-1),'price':row['curr_price'] },ignore_index=True)
#     else:
#         for ind,row in ideal.iterrows():
#             order_df=order_df.append({'symb':row['symb'],'qty':row['qty'],'price':row['price'] },ignore_index=True)  
    return order_df
def place_order(data):
    for index, row in data.iterrows():
        if row["qty"]>0:
            api.submit_order(
              symbol=row["symb"], # Replace with the ticker of the stock you want to buy
              #notional=str(notional),
              qty=row["qty"],
              side='buy',
              type='limit', 
              limit_price=str(row["price"]),
              time_in_force='day'
            )
        else: 
             api.submit_order(
              symbol=row["symb"], # Replace with the ticker of the stock you want to buy
              #notional=str(notional),
              qty=-row["qty"],
              side='sell',
              type='limit', 
              limit_price=str(row["price"]),
              time_in_force='day' 
             )
                 
def get_latest_price(symb_list):
    latest_list={}
    latest_data=api.get_latest_bars(symb_list)
    for symb in symb_list:
        datapoint=latest_data[symb]._raw
        price=datapoint['l']
        latest_list[symb]=price
    return latest_list
def get_orders_status():
    orders=api.list_orders()
    avg=[]
    symbol=[]
    qty=[]
    filled_qty=[]
    limit_price=[]
    for order in orders:
        order=order._raw
        avg.append(order['filled_avg_price'])
        symbol.append(order['symbol'])
        qty.append(order['qty'])
        filled_qty.append(order['filled_qty'])
        limit_price.append(order['limit_price'])
    latest_list=get_latest_price(symbol)
    order_data=pd.DataFrame(list(zip(symbol,qty,filled_qty,avg,limit_price)),columns=['symb','qty','filled_qty','avg','price'])
    #order_data=pd.DataFrame(list(zip(symbol,qty,filled_qty,avg,latest_list)),columns=['symb','qty','filled_qty','avg','price'])
    #latest_price=api.get_latest_bars(symbol)   
    return order_data 



def cancel_all_orders():
    api.cancel_all_orders()       

In [5]:
data=get_data(symb_list,TimeFrame.Day, "2022-07-29",  "2022-08-03", limit=9999)
sorted_data=sort_data(data)
sorted_data['qty']=(2000/sorted_data['price']).astype(int)
ideal_position=get_ideal_position(sorted_data,5)
curr_position=get_curr_position()
price_list=get_latest_price(symb_list)

In [42]:
def generate_order_df(ideal,curr,price):
    
    order_df=pd.DataFrame({'symb':[],'qty':[],'price':[]})
    print(list(ideal['symb']))
    print(curr['symb'])
    for ind,row in ideal.iterrows():

        if row['symb'] in list(curr['symb']):
            print(row['symb'],1)
            index=curr.index[curr['symb']==row['symb']]
            order_df=order_df.append({'symb':row['symb'],'qty':(int(row['qty'])-int(curr['qty'][index])),'price':row['price'] },ignore_index=True)
        else:
            print(row['symb'],2)
            order_df=order_df.append({'symb':row['symb'],'qty':row['qty'],'price':row['price'] },ignore_index=True)  
    print(ideal['symb'])
    for ind,row in curr.iterrows():
        
        if row['symb'] not in list(ideal['symb']):
            print(row['symb'])
            order_df=order_df.append({'symb':row['symb'],'qty':(int(row['qty'])*-1),'price':row['curr_price'] },ignore_index=True)
#     else:
#         for ind,row in ideal.iterrows():
#             order_df=order_df.append({'symb':row['symb'],'qty':row['qty'],'price':row['price'] },ignore_index=True)  
    return order_df

In [62]:

print(ideal_position)

print(curr_position)

   symb   price    change  qty
0  DDOG  112.42  0.098150   17
1  DOCU   71.12  0.106718   28
2  LCID   20.56  0.119891   97
3  PYPL   97.92  0.125633   20
4  MRNA  186.49  0.133155   10
   symb  qty           entry_price            curr_price    gain
0  DOCU  140                70.214                70.214  297.64
1  DDOG   51  108.8766666666666667  108.8766666666666667  210.29
2  PYPL   60                 96.89                 96.89 -103.80
3  LCID  291                 18.56                 18.56 -157.14


In [68]:
df_2 = ideal_position.merge(curr_position[["symb", "qty"]], on = "symb", how = "outer")
df_2.rename(columns={"qty_x" : "1"}, inplace=True)
df_2

,symb,price,change,1,qty_y
0,DDOG,112.42,0.098150,17,51
1,DOCU,71.12,0.106718,28,140
2,LCID,20.56,0.119891,97,291
3,PYPL,97.92,0.125633,20,60
4,MRNA,186.49,0.133155,10,NaN


In [37]:
curr_position.index[curr_position['symb']=="DDOG"]
curr_position['qty'][1]

'60'

In [40]:

order_to_be_placed=generate_order_df(ideal_position,curr_position,price_list)
print(order_to_be_placed)
cancel_all_orders()


['DDOG', 'DOCU', 'LCID', 'PYPL', 'MRNA']
0    DDOG
1    PYPL
2    LCID
3    DOCU
Name: symb, dtype: object
DDOG 1
DOCU 1
LCID 1
PYPL 1
MRNA 2
0    DDOG
1    DOCU
2    LCID
3    PYPL
4    MRNA
Name: symb, dtype: object
   symb   qty   price
0  DDOG   -34  112.42
1  DOCU  -112   71.12
2  LCID  -194   20.56
3  PYPL   -40   97.92
4  MRNA    10  186.49


In [61]:
def weekly_update(curr):
    print(curr)
    curr['gain']=curr['qty'].astype(float)*(pd.to_numeric(curr['curr_price'],errors='coerce')-pd.to_numeric(curr['entry_price'],errors='coerce'))
    print(curr)
    curr['curr_price']=curr['entry_price']
weekly_update(curr_position)

   symb  qty           entry_price curr_price    gain
0  DOCU  140                70.214      72.34  297.64
1  DDOG   51  108.8766666666666667        113  210.29
2  PYPL   60                 96.89      95.16 -103.80
3  LCID  291                 18.56      18.02 -157.14
   symb  qty           entry_price curr_price    gain
0  DOCU  140                70.214      72.34  297.64
1  DDOG   51  108.8766666666666667        113  210.29
2  PYPL   60                 96.89      95.16 -103.80
3  LCID  291                 18.56      18.02 -157.14


In [43]:
cancel_all_orders()
order_history=[]
for i in range(5):
    order_to_be_placed=generate_order_df(ideal_position,curr_position,price_list)
    print(order_to_be_placed)
    place_order(order_to_be_placed)
    time.sleep(5)
    order_history.append(get_orders_status)
    cancel_all_orders()
    curr_position=get_curr_position()
    

['DDOG', 'DOCU', 'LCID', 'PYPL', 'MRNA']
0    DDOG
1    PYPL
2    LCID
3    DOCU
Name: symb, dtype: object
DDOG 1
DOCU 1
LCID 1
PYPL 1
MRNA 2
0    DDOG
1    DOCU
2    LCID
3    PYPL
4    MRNA
Name: symb, dtype: object
   symb    qty   price
0  DDOG  -34.0  112.42
1  DOCU -112.0   71.12
2  LCID -194.0   20.56
3  PYPL  -40.0   97.92
4  MRNA   10.0  186.49
['DDOG', 'DOCU', 'LCID', 'PYPL', 'MRNA']
0    DOCU
1    DDOG
2    PYPL
3    LCID
Name: symb, dtype: object
DDOG 1
DOCU 1
LCID 1
PYPL 1
MRNA 2
0    DDOG
1    DOCU
2    LCID
3    PYPL
4    MRNA
Name: symb, dtype: object
   symb    qty   price
0  DDOG  -34.0  112.42
1  DOCU -112.0   71.12
2  LCID -194.0   20.56
3  PYPL  -40.0   97.92
4  MRNA   10.0  186.49
['DDOG', 'DOCU', 'LCID', 'PYPL', 'MRNA']
0    DOCU
1    DDOG
2    PYPL
3    LCID
Name: symb, dtype: object
DDOG 1
DOCU 1
LCID 1
PYPL 1
MRNA 2
0    DDOG
1    DOCU
2    LCID
3    PYPL
4    MRNA
Name: symb, dtype: object
   symb    qty   price
0  DDOG  -34.0  112.42
1  DOCU -112.0   71.12
2

In [ ]:
def daily_update(curr):
    print(curr)
    curr['gain']=curr['qty'].astype(float)*(pd.to_numeric(curr['curr_price'],errors='coerce')-pd.to_numeric(curr['entry_price'],errors='coerce'))
    print(curr)
    curr['entry_price']=curr['curr_price']
111#weekly_update(curr_position)
# big dataframe 
#inquiry by symb/date/side
def interrupt(curr_position,gain_thres,loss_thres):
    for ind,row in curr_position.iterrows():
        if row['curr_price']<row['entry_price']*(1-loss_thres):
            api.submit_order(
              symbol=row["symb"], 
              qty=row["qty"],
              side='sell',
              type='market', 
              #limit_price=str(row["price"]),
              time_in_force='day'
            )   
            
        elif row['curr_price']>row['entry_price']*(1+gain_thres):
             api.submit_order(
              symbol=row["symb"], 
              qty=row["qty"],
              side='sell',
              type='market', 
              #limit_price=str(row["price"]),
              time_in_force='day'
            )          

In [ ]:
curr_position

In [ ]:
# subset=sorted_data.loc[sorted_data['symb'].isin(curr_position['symb'])]
# subset=subset.sort_values(by=['symb'])
# subset = subset.reset_index(drop=True)
# curr_position=curr_position.sort_values(by=['symb'])
# subset = subset.reset_index(drop=True)
order_df=pd.DataFrame({'symb':[],'qty':[],'price':[]})
df2=pd.DataFrame(columns=["f","u","c","k"])
print(df2)
# copy=ideal_position.copy()
# print(copy)


In [ ]:
print(ideal_position)

print(curr_position)

In [ ]:
for ind,row in ideal_position.iterrows():
    
    if row['symb'] in curr_position["symb"]:
        order_df.append({'symb':row['symb'],'qty':row['qty']-curr_position['qty'],'price':row['price'] })
    else:
        order_df=order_df.append({'symb':row['symb'],'qty':row['qty'],'price':row['price'] },ignore_index=True)
for ind,row in curr_position.iterrows():
    if row['symb'] not in ideal_position['symb']:
        order_df=order_df.append({'symb':row['symb'],'qty':str(int(row['qty'])*-1),'price':row['curr_price'] },ignore_index=True)
print(order_df)

In [ ]:
print(order_df)

In [ ]:
account=api.get_account()
#print(account)
?api.get_latest_trades

In [ ]:
print(sorted_data)

In [ ]:
trades=api.get_latest_trades([''])
trades

In [ ]:
trades=api.get_latest_trades(['CEG'])
trades

In [19]:
?api.get_trades

In [20]:
333#convert trading to dataframe

[Trade({   'S': 'LCID',
     'c': ['@', 'T', 'I'],
     'i': 582,
     'p': 18.13,
     's': 10,
     't': '2022-08-04T04:01:42.267Z',
     'x': 'D',
     'z': 'C'}),
 Trade({   'S': 'LCID',
     'c': ['@', 'T', 'I'],
     'i': 643,
     'p': 18.13,
     's': 1,
     't': '2022-08-04T04:02:32.707Z',
     'x': 'D',
     'z': 'C'}),
 Trade({   'S': 'LCID',
     'c': ['@', 'T', 'I'],
     'i': 600,
     'p': 18.12,
     's': 9,
     't': '2022-08-04T04:09:51.688Z',
     'x': 'D',
     'z': 'C'}),
 Trade({   'S': 'LCID',
     'c': ['@', 'T', 'I'],
     'i': 651,
     'p': 18.12,
     's': 8,
     't': '2022-08-04T04:13:18.008Z',
     'x': 'D',
     'z': 'C'}),
 Trade({   'S': 'LCID',
     'c': ['@', 'T'],
     'i': 638,
     'p': 18.12,
     's': 100,
     't': '2022-08-04T04:15:32.211Z',
     'x': 'D',
     'z': 'C'}),
 Trade({   'S': 'LCID',
     'c': ['@', 'T', 'I'],
     'i': 616,
     'p': 18.12,
     's': 1,
     't': '2022-08-04T04:15:59.747Z',
     'x': 'D',
     'z': 'C'}),
 Trade

In [24]:
api.get_activities()

[AccountActivity({   'activity_type': 'FILL',
     'cum_qty': '34',
     'id': '20220804125440502::e5ad34c8-80d4-45f1-92ff-c5b083c7b0b5',
     'leaves_qty': '0',
     'order_id': 'e92fc13b-face-4550-8c73-db3f4f6978a0',
     'order_status': 'filled',
     'price': '108.77',
     'qty': '34',
     'side': 'sell',
     'symbol': 'DDOG',
     'transaction_time': '2022-08-04T16:54:40.502697Z',
     'type': 'fill'}),
 AccountActivity({   'activity_type': 'FILL',
     'cum_qty': '40',
     'id': '20220804125435956::5b8bf442-c00a-49ea-aaa3-b30d37813d65',
     'leaves_qty': '0',
     'order_id': 'd72de1b1-08e1-4d1c-927c-5e0c16e86653',
     'order_status': 'filled',
     'price': '96.97',
     'qty': '40',
     'side': 'sell',
     'symbol': 'PYPL',
     'transaction_time': '2022-08-04T16:54:35.956337Z',
     'type': 'fill'}),
 AccountActivity({   'activity_type': 'FILL',
     'cum_qty': '20',
     'id': '20220804125435117::d239c960-0160-49b6-a2d3-3ca9377f3470',
     'leaves_qty': '0',
     'ord

In [4]:
api.get_account()#222 4pm everyday

Account({   'account_blocked': False,
    'account_number': 'PA3YSIK3R9QS',
    'accrued_fees': '3.7848184722222220947336',
    'buying_power': '374397.08',
    'cash': '74785.27',
    'created_at': '2022-08-02T07:07:33.664479Z',
    'crypto_status': 'ACTIVE',
    'currency': 'USD',
    'daytrade_count': 45,
    'daytrading_buying_power': '374397.08',
    'equity': '101629.29',
    'id': '121ac014-6975-4bc4-8888-1ed671fb66e6',
    'initial_margin': '13422.01',
    'last_equity': '101662.41',
    'last_maintenance_margin': '8063.14',
    'long_market_value': '26844.02',
    'maintenance_margin': '8053.206',
    'multiplier': '4',
    'non_marginable_buying_power': '86207.28',
    'pattern_day_trader': True,
    'pending_transfer_in': '0',
    'portfolio_value': '101629.29',
    'regt_buying_power': '176414.56',
    'short_market_value': '0',
    'shorting_enabled': True,
    'sma': '101606.62',
    'status': 'ACTIVE',
    'trade_suspended_by_user': False,
    'trading_blocked': False,
 